In [11]:
import os
import torch
import numpy as np
from tqdm import tqdm
from scipy.ndimage import uniform_filter

In [ ]:
lead_time = 6
base_dir = f"/gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t{lead_time}"

target_hour = "12"
windows = [3, 9, 25, 49, 81, 121]
PIXEL_SIZE_KM = 3

In [13]:
def compute_fss(pred, obs, window):
    pred = np.clip(pred, 0, 1)
    obs = np.clip(obs, 0, 1)
    f_pred = uniform_filter(pred, size=window, mode="constant")
    f_obs = uniform_filter(obs, size=window, mode="constant")
    num = np.mean((f_pred - f_obs) ** 2)
    den = np.mean(f_pred ** 2 + f_obs ** 2)
    return 1 - num / (den + 1e-8)

In [14]:
all_files = sorted(f for f in os.listdir(base_dir) if f.endswith(".pt"))
filtered_files = [f for f in all_files if f[9:11] == target_hour]
print(f"Found {len(filtered_files)} files at hour={target_hour} UTC")

fss_results = {w: {"zcast": [], "nflics": [], "netncc": []} for w in windows}

for f in tqdm(filtered_files, desc="Computing FSS"):
    file_path = os.path.join(base_dir, f)
    try:
        data = torch.load(file_path, weights_only=False)
    except EOFError:
        print(f"Skipping corrupted file: {file_path}")
        continue
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        continue

    gt = data["ground_truth"].astype(np.float32)
    zcast = data["zcast"].astype(np.float32)
    nflics = data["nflics"].astype(np.float32) / 100
    netncc = data["netncc"].astype(np.float32) / 100

    for w in windows:
        fss_results[w]["zcast"].append(compute_fss(zcast, gt, w))
        fss_results[w]["nflics"].append(compute_fss(nflics, gt, w))
        fss_results[w]["netncc"].append(compute_fss(netncc, gt, w))

print(f"\nAverage FSS for hour={target_hour} UTC (lead t+{lead_time})")
print(f"{'Window':>8} | {'Scale(km)':>10} | {'ZCAST':>10} | {'NFLICS':>10} | {'NetNCC':>10}")
print("-" * 62)

for w in windows:
    scale = w * PIXEL_SIZE_KM
    mean_z = np.nanmean(fss_results[w]["zcast"])
    mean_n = np.nanmean(fss_results[w]["nflics"])
    mean_c = np.nanmean(fss_results[w]["netncc"])
    print(f"{w:>8} | {scale:>10.0f} | {mean_z:.4f} | {mean_n:.4f} | {mean_c:.4f}")


Found 537 files at hour=12 UTC


Computing FSS:  29%|██▉       | 157/537 [00:15<00:16, 23.39it/s]

Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201202_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201202_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201202_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201202_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201203_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201203_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201203_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201203_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/I

Computing FSS:  31%|███       | 166/537 [00:15<00:12, 30.40it/s]

Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201204_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201204_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201204_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201205_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201205_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201205_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201205_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20201206_1200.pt


...

Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211213_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211214_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211214_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211214_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211214_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211215_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211215_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211215_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/I

Computing FSS:  98%|█████████▊| 524/537 [00:24<00:00, 42.18it/s]

Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211217_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211217_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211217_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211218_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211218_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211218_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211218_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211221_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/I

Computing FSS: 100%|██████████| 537/537 [00:24<00:00, 21.91it/s]

Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211222_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211222_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211222_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211222_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211223_1200.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211223_1230.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211223_1245.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t6/20211224_1215.pt
Skipping corrupted file: /gws/nopw/j04/wiser_ewsa/mrakotomanga/I